## Data Cleaning Notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
path = "../ift6758/data/JSON/example.json"
with open(path) as f:
    data1 = json.load(f)


In [3]:
from  ift6758.data.data_acquisition import Season
Season2017 = Season(2017,"../ift6758/data")
# df[["gamePk","liveData.plays.allPlays"]]




Len of games_list in 2017 is 1488


In [195]:
Season2017.clean_data()

AttributeError: 'Season' object has no attribute 'clean_data'

In [138]:
## Reference on how to use json_normalize: https://pandas.pydata.org/pandas-docs/version/1.2.0/reference/api/pandas.json_normalize.html
df1 = pd.json_normalize(Season2017.data[:10],record_path=['liveData','plays','allPlays'],meta=['gamePk'])
## To add game time
df1.head()


,result.event,result.eventCode,result.eventTypeId,result.description,about.eventIdx,about.eventId,about.period,about.periodType,about.ordinalNum,about.periodTime,...,team.link,team.triCode,result.secondaryType,result.penaltySeverity,result.penaltyMinutes,result.strength.code,result.strength.name,result.gameWinningGoal,result.emptyNet,gamePk
0,Game Scheduled,LAK1,GAME_SCHEDULED,Game Scheduled,0,1,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017010001
1,Period Ready,LAK54,PERIOD_READY,Period Ready,1,54,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017010001
2,Period Start,LAK2,PERIOD_START,Period Start,2,2,1,REGULAR,1st,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017010001
3,Faceoff,LAK3,FACEOFF,Markus Granlund faceoff won against Adrian Kempe,3,3,1,REGULAR,1st,00:00,...,/api/v1/teams/23,VAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017010001
4,Penalty,LAK57,PENALTY,Adrian Kempe Hooking against Philip Holm,4,57,1,REGULAR,1st,00:19,...,/api/v1/teams/26,LAK,Hooking,Minor,2.0,NaN,NaN,NaN,NaN,2017010001


In [192]:
## !!?? Add Game time
##? Get IDs of players instead of names
import numpy as np
def important_players(players):
    # "[{'player': {'id': 8475790, 'fullName': 'Erik Gudbranson', 'link': '/api/v1/people/8475790'}, 'playerType': 'Shooter'}, {'player': {'id': 8471734, 'fullName': 'Jonathan Quick', 'link': '/api/v1/people/8471734'}, 'playerType': 'Goalie'}]"
    shooter = np.NaN
    goalie = np.NaN
    for player in players:
        playerType = player.get("playerType", np.NaN)
        if playerType == "Shooter" or playerType== "Scorer":
            shooter = player["player"].get("fullName", np.NaN)
        elif playerType == "Goalie":
            goalie = player["player"].get("fullName", np.NaN)
    return shooter,goalie


select_columns = ["result.event","gamePk","team.name","players","about.period","about.periodTime","about.periodType","about.periodTimeRemaining","coordinates.x","coordinates.y","result.secondaryType","result.emptyNet","result.strength.name"]

df2 = df1[select_columns]
df3=df2[(df2["result.event"]=="Shot")| (df2["result.event"]=="Goal")]
df3['shooter'],df3['goalie'] = zip(*df3["players"].map(important_players))
df4 = df3.drop(columns=["players"],axis=1).reset_index(drop=True)
df4.head()



C:\Users\saiak\AppData\Local\Temp/ipykernel_23500/3156143219.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['shooter'],df3['goalie'] = zip(*df3["players"].map(important_players))


,result.event,gamePk,team.name,about.period,about.periodTime,about.periodType,about.periodTimeRemaining,coordinates.x,coordinates.y,result.secondaryType,result.emptyNet,result.strength.name,shooter,goalie
0,Shot,2017010001,Vancouver Canucks,1,05:29,REGULAR,14:31,-43.0,5.0,Wrist Shot,NaN,NaN,Erik Gudbranson,Jonathan Quick
1,Shot,2017010001,Vancouver Canucks,1,05:54,REGULAR,14:06,75.0,-13.0,Snap Shot,NaN,NaN,Markus Granlund,Jonathan Quick
2,Shot,2017010001,Vancouver Canucks,1,05:56,REGULAR,14:04,84.0,-9.0,Wrist Shot,NaN,NaN,Markus Granlund,Jonathan Quick
3,Shot,2017010001,Los Angeles Kings,1,06:09,REGULAR,13:51,-70.0,1.0,Wrist Shot,NaN,NaN,Tyler Toffoli,Jacob Markstrom
4,Shot,2017010001,Los Angeles Kings,1,06:40,REGULAR,13:20,-54.0,-25.0,Slap Shot,NaN,NaN,Oscar Fantenberg,Jacob Markstrom


In [189]:
df4.to_csv("dummy.csv",index=False)